In [144]:
import numpy as np

import random
from collections import deque
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

import torch.optim as optim

import cv2

import game

from IPython.display import clear_output

import random
from collections import deque

In [176]:
from importlib import reload
reload(game)

<module 'game' from '/home/ari/Documents/driving_simulator/game.py'>

In [93]:
num_states = 100
hidden_size = 100
num_actions = 2

In [100]:
env = game.game()

In [95]:
pygame.display.quit()

In [101]:
conv = Conv(4)

In [102]:
obs, reward, done = env.step(1, 1, np.random.rand(80, 80))

In [103]:
obs.shape

(80, 80)

In [104]:
x_t = np.stack((obs, obs, obs, obs), axis = 0).astype(np.float64)

In [105]:
x = x_t[np.newaxis, :]

In [106]:
x.shape

(1, 4, 80, 80)

In [107]:
res = conv(torch.tensor(x, dtype=torch.float))

In [108]:
res.shape

torch.Size([1, 6400])

In [114]:
actor = Actor(6400, 256, 2)
actor_target = Actor(6400, 256, 2)

In [115]:
critic = Critic(6402, 256, 2)
critic_target = Critic(6402, 256, 2)

In [116]:
for target_param, param in zip(actor_target.parameters(), actor.parameters()):
    target_param.data.copy_(param.data)
for target_param, param in zip(critic_target.parameters(), critic.parameters()):
    target_param.data.copy_(param.data)

In [120]:
action = actor.forward(res)

In [121]:
action

tensor([[-0.0872,  0.0016]], grad_fn=<TanhBackward>)

In [122]:
action.shape

torch.Size([1, 2])

In [124]:
res.shape

torch.Size([1, 6400])

In [130]:
augment = torch.cat((res, action), axis = 1).shape

In [132]:
critic.forward(res, action)

tensor([[-0.0353, -0.0239]], grad_fn=<AddmmBackward>)

In [243]:
class DDPG_agent:
    
    def __init__(self, actor_learning_rate=1e-4, critic_learning_rate=1e-4, batch_size=64, tau=0.001, gamma=0.0001):
        
        self.batch_size = batch_size
        
        self.gamma = gamma
        self.tau = tau
        
        self.actor = Actor(6400, 254, 2)
        self.actor_target = Actor(6400, 254, 2)
        
        self.critic = Critic(6402, 254, 2)
        self.critic_target = Critic(6402, 254, 2)
        
        self.conv = Conv(4)
        
        self.memory = Memory(10000)
        
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data)

        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data)
            
        self.critic_criterion  = nn.MSELoss()
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=actor_learning_rate)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_learning_rate)
        
    def get_action(self, state):
        embed = self.conv.forward(state)
        action = self.actor.forward(embed).detach().numpy()
        return action
        
    def update(self):
        
        states, actions, rewards, next_states, _ = self.memory.sample(self.batch_size)
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        
        print(np.array(next_states).shape)
        next_states = torch.FloatTensor(next_states)
        
        states = self.conv.forward(states)
        next_states = self.conv.forward(next_states)
        
        print(states.shape)
        print(actions.shape)
        
        Qvals = self.critic.forward(states, actions)
        next_actions = self.actor_target.forward(next_states)
        next_Q = self.critic_target.forward(next_states, next_actions.detach())
        Qprime = rewards + self.gamma * next_Q
        
        critic_loss = self.critic_criterion(Qvals, Qprime)
        
        policy_loss = -self.critic.forward(states, self.actor.forward(states)).mean()
        
        # update networks
        self.actor_optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)
        self.actor_optimizer.step()

        self.critic_optimizer.zero_grad()
        critic_loss.backward() 
        self.critic_optimizer.step()

        # update target networks 
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
       
        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))

In [244]:
agent = DDPG_agent()

In [245]:
np.random.rand(1, 4, 80, 80).squeeze().shape

(4, 80, 80)

In [246]:
noise = OUNoise()
batch_size = 64
rewards = []
avg_rewards = []

for episode in range(50):
    state = env.reset()
    noise.reset()
    episode_reward = 0
    
    state = np.stack((state, state, state, state), axis = 0)[np.newaxis, :]
    
    
    for step in range(500):
        action = agent.get_action(torch.FloatTensor(state))
        action = noise.get_action(action, step)
        #print(action[0, 0])
        new_state, reward, done = env.step(action[0, 0], 1, np.random.rand(80,80))
        
        #print(state.shape)
        new_state = new_state[np.newaxis, np.newaxis, :, :]
        new_state = np.append(new_state, state[:,:3,:,:], axis=1)
        
        print(np.array(new_state.shape))
        
        agent.memory.push(state.squeeze(), action.squeeze(), reward, new_state.squeeze(), done)
        
        
        if len(agent.memory) > batch_size:
            agent.update()        
        
        
        state = new_state
        episode_reward += reward

        if done:
            print("episode: {}, reward: {}, average _reward: {} \n".format(episode, np.round(episode_reward, decimals=2), np.mean(rewards[-10:])))
            break


[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
episode: 0, reward: 29.02, average _reward: nan 

[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
episode: 1, reward: 136.89, average _reward: nan 

[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
episode: 2, reward: 25.09, average _reward: nan 

[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80 80]
[ 1  4 80

KeyboardInterrupt: 

In [232]:
for e in range(episodes):
    print("Episode {}".format(e))
    
    obs = env.reset()
    
    done = False
    
    x_t = embedding.forward(obs)
    
    s_t = np.stack((x_t, x_t, x_t, x_t), axis = 2)
    
    while not done:
        
        steering, throttle = actor.get_action()
        
        new_obs, reward, done = env.step(steering, throttle, x_t.T)
        
        x_t = embedding.forward(new_obs)
        
        s_t1 = np.append(x_t1, s_t[:,:,:3], axis=2)
        
        mem.push(s_t, np.array([steering, throttle]), reward, s_t1, done)
        
        if train:
            train()
            
        s_t = s_t1
        
        

NameError: name 'episodes' is not defined

In [11]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Conv(nn.Module):
    def __init__(self, img_channels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, 24, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(24, 32, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            Flatten(),
        )
    def forward(self, x):
        #print(x.shape)
        #inp = x.reshape((64, 1, 80, 80))
        #inp = x.unsqueeze(0)
        return self.net(x)

In [238]:
class Critic(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Critic, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, state, action):
        """
        Params state and actions are torch tensors
        """
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x

In [111]:
class Actor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 3e-4):
        super(Actor, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, state):
        """
        Param state is a torch tensor
        """
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = torch.tanh(self.linear3(x))

        return x

In [141]:
class Memory:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)
    
    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)
        
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch

    def __len__(self):
        return len(self.buffer)

In [29]:
states = embedding.forward(states)
next_sates = embedding.forward(next_states)
Qvals = critic.forward(states, actions)
next_actions = actor_target.forward(next_states)
next_Q = critic_target.forward(next_states, next_actions.detach())
Qprime = rewards + gamma * next_Q

critic_loss = nn.MSELoss(Qvals, Qprime)
critic_optimizer.zero_grad()
critic_loss.backward() 
critic_optimizer.step()

policy_loss = -critic.forward(states, actor.forward(states)).mean()

actor_optimizer.zero_grad()
policy_loss.backward()
actor_optimizer.step()

for target_param, param in zip(actor_target.parameters(), actor.parameters()):
    target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)
       
for target_param, param in zip(critic_target.parameters(), critic.parameters()):
    target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)

NameError: name 'states' is not defined

In [147]:
np.array([1,2,3])

array([1, 2, 3])

In [148]:
_.high

AttributeError: 'numpy.ndarray' object has no attribute 'high'

In [152]:
class OUNoise():
    def __init__(self, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.3, decay_period=100000):
        self.mu           = mu
        self.theta        = theta
        self.sigma        = max_sigma
        self.max_sigma    = max_sigma
        self.min_sigma    = min_sigma
        self.decay_period = decay_period
        self.action_dim   = 2
        self.low          = -0.1
        self.high         = 0.1
        self.reset()
        
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
        
    def evolve_state(self):
        x  = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)
        self.state = x + dx
        return self.state
    
    def get_action(self, action, t=0): 
        ou_state = self.evolve_state()
        self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
        return np.clip(action + ou_state, self.low, self.high)